In [1]:
import time
from joblib import Parallel, delayed
from scipy.io import loadmat 
import numpy as np
from pandas import read_csv
from multiprocessing import Process, Queue
from math import floor


In [2]:

# filename = 'Mug128_singlepixcam.mat'
filename = 'Mug32_singlepixcam.mat'
array = loadmat(filename)

X = np.array(array['A'])
Y = np.array(array['y'])
nr,nc = X.shape
# print X.shape
beta = 1;  # For Squared Loss
lamda = 0.216;

#Normalization-1
minimum = X.min(0);
maximum = X.max(0);

for i in np.arange(nc):
    if(maximum[i]!=minimum[i]):
        X[:,i] = (X[:,i]-minimum[i]) / (maximum[i]-minimum[i]);
    else:
        X[:,i] = np.ones(nr)

#Normalization-2
X = X - X.mean(axis=0)
nrm = np.linalg.norm(X,axis = 0);

for i in np.arange(nc):
	if nrm[i]>0:
		X[:,i] = X[:,i]/nrm[i]

Data = X
X = np.concatenate((X,-X), axis = 1)
nc = 2*nc;
# print nc,X.shape
# print nc
# sys.exit(0)

w = np.zeros(nc,dtype = np.float64).reshape(nc,1) +1
# w = np.random.rand(nc,1)

def func(index):
	# print index
	t1 = np.dot(X,w)
	t2 = t1- Y

	g = np.dot(X[:,index].T,t2) + lamda; #why nr??
	w[index] = w[index] -   0.001*g/beta; # alpha
	# print w[index]
	if w[index]<0:
		w[index] = 0;

	return w[index]
    
for i in np.arange(5):
    for k in np.arange(0,nc-100,100):
        with Parallel(n_jobs=20) as parallel:
            results = parallel(delayed(func)(j)for j in np.arange(k,k+100) )
        w[k:k+100] = results
    
    
    print (i)
    temp = nc/2
    z1 = np.subtract(w[temp:nc] , w[0:temp])
    z1 = z1.reshape(temp,1)
    
    z2 = np.subtract(w[0:temp],w[temp:nc])
    z2 = z2.reshape(temp,1)
    # pred = np.dot(X , w)#.reshape(nr,1);    Change W acc to algo
    pred = np.dot(Data,z1)
    differ = (pred-Y)
    MSE = np.mean( np.square(differ) )
    print ('z1 w2-w1 MSE Error:%f' % MSE)

    pred = np.dot(Data,z2)
    differ = (pred-Y)
    MSE = np.mean( np.square(differ) )
    print ('z2  w1-w2  MSE Error:%f' % MSE)
            
    print (np.linalg.norm(w))

# z1 w2-w1 MSE Error:0.002479
# z2  w1-w2  MSE Error:0.000189
# 43.0451249322
# 232
# z1 w2-w1 MSE Error:0.002483
# z2  w1-w2  MSE Error:0.000189
# 43.035619945
# 233
# z1 w2-w1 MSE Error:0.002487
# z2  w1-w2  MSE Error:0.000189
# 43.0261151329
# 234
# z1 w2-w1 MSE Error:0.002491
# z2  w1-w2  MSE Error:0.000189
# 43.0166104262
# 235

0
z1 w2-w1 MSE Error:0.000785
z2  w1-w2  MSE Error:0.000764
45.2452837335


/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:63: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:64: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:66: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:67: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1
z1 w2-w1 MSE Error:0.000796
z2  w1-w2  MSE Error:0.000754
45.2357338011
2
z1 w2-w1 MSE Error:0.000806
z2  w1-w2  MSE Error:0.000744
45.22618417
3
z1 w2-w1 MSE Error:0.000817
z2  w1-w2  MSE Error:0.000734
45.2166347992
4
z1 w2-w1 MSE Error:0.000827
z2  w1-w2  MSE Error:0.000724
45.2070858034


In [3]:
temp = nc/2
z = w[temp:nc] - w[0:temp]
z = z.reshape(temp,1)
# pred = np.dot(X , w)#.reshape(nr,1);    Change W acc to algo
pred = np.dot(Data,z)
differ = (pred-Y)
MSE = np.mean( np.square(differ) );
print ('MSE Error:%f' % MSE)

MSE Error:0.000827


/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
/home/clabuser/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
